In [1]:
from itertools import product

import pandas as pd
from SALib import ProblemSpec
import numpy as np
from plotly.subplots import make_subplots

import plotly.graph_objects as go

In [2]:
cost_function_results = pd.read_parquet("./output_sobol_index.parquet")
cost_function_results

station                BARENTS                            HOT                \
quantity_of_interest      mean      variance argmax      mean      variance   
0                     0.004000  3.479870e-06  205.0  0.002950  2.277094e-07   
1                     0.000044  4.303894e-10  205.0  0.000033  2.816304e-11   
2                     0.003988  3.623789e-06  208.0  0.002950  2.277094e-07   
3                     0.003999  3.456675e-06  205.0  0.002950  2.277094e-07   
4                     0.008151  5.712565e-06  254.0  0.006751  5.638046e-07   
...                        ...           ...    ...       ...           ...   
476995                0.000235  1.172422e-07  131.0  0.000107  5.361083e-10   
476996                0.000234  1.195310e-07  131.0  0.000107  5.361083e-10   
476997                0.000814  8.356571e-07  151.0  0.000107  9.215270e-10   
476998                0.000512  3.765561e-07  148.0  0.000114  1.009265e-09   
476999                0.000224  1.068438e-07  142.0  0.000107  9.215270e-10   

station                          BATS                           PAPA  \
quantity_of_interest argmax      mean      variance argmax      mean   
0                      50.0  0.003824  2.358212e-06   97.0  0.006814   
1                      50.0  0.000043  2.916630e-10   97.0  0.000076   
2                      50.0  0.003824  2.358212e-06   97.0  0.006795   
3                      50.0  0.003824  2.358212e-06   97.0  0.006810   
4                      79.0  0.008491  4.738705e-06  111.0  0.014415   
...                     ...       ...           ...    ...       ...   
476995                 48.0  0.000120  4.139368e-09   77.0  0.000244   
476996                 48.0  0.000120  3.721616e-09   91.0  0.000244   
476997                 43.0  0.000120  3.997736e-09   92.0  0.000806   
476998                 43.0  0.000148  6.845959e-09   92.0  0.000708   
476999                 43.0  0.000120  3.997731e-09   92.0  0.000242   

station                                        GUAM                       
quantity_of_interest      variance argmax      mean      variance argmax  
0                     7.913743e-07  287.0  0.001023  6.137193e-09  113.0  
1                     9.787696e-11  287.0  0.000011  7.590464e-13  113.0  
2                     9.268995e-07  289.0  0.001023  6.137193e-09  113.0  
3                     8.046640e-07  289.0  0.001023  6.137193e-09  113.0  
4                     2.375407e-06  300.0  0.002324  1.664002e-08  113.0  
...                            ...    ...       ...           ...    ...  
476995                9.989848e-09  108.0  0.000047  4.177072e-11  196.0  
476996                1.047774e-08  108.0  0.000047  4.177072e-11  196.0  
476997                1.144070e-07  119.0  0.000047  4.177072e-11  196.0  
476998                4.240498e-08  119.0  0.000047  4.181343e-11  196.0  
476999                1.041714e-08  137.0  0.000047  4.177072e-11  196.0  

[479000 rows x 15 columns]

In [3]:
cost_function_results.describe().transpose().reset_index().groupby(["quantity_of_interest", "station"]).mean()

count          mean           std  \
quantity_of_interest station                                         
argmax               BARENTS  479000.0  1.896562e+02  4.177737e+01   
                     BATS     479000.0  9.752660e+01  1.647366e+01   
                     GUAM     479000.0  1.636027e+02  7.027309e+01   
                     HOT      479000.0  7.155415e+01  5.494519e+01   
                     PAPA     479000.0  1.904802e+02  7.758618e+01   
mean                 BARENTS  479000.0  5.216743e-03  5.828877e-03   
                     BATS     479000.0  3.262964e-03  8.344485e-03   
                     GUAM     479000.0  1.015853e-03  3.047637e-03   
                     HOT      479000.0  2.719892e-03  7.591823e-03   
                     PAPA     479000.0  7.484984e-03  1.018934e-02   
variance             BARENTS  479000.0  9.217323e-06  1.196880e-05   
                     BATS     479000.0  2.783155e-06  7.734698e-06   
                     GUAM     479000.0  4.716149e-08  3.975885e-07   
                     HOT      479000.0  3.617294e-07  1.514838e-06   
                     PAPA     479000.0  2.928902e-06  6.631183e-06   

                                       min           25%           50%  \
quantity_of_interest station                                             
argmax               BARENTS  1.150000e+02  1.470000e+02  2.000000e+02   
                     BATS     1.400000e+01  9.100000e+01  9.400000e+01   
                     GUAM     2.000000e+00  1.130000e+02  1.960000e+02   
                     HOT      1.000000e+01  4.800000e+01  4.800000e+01   
                     PAPA     6.400000e+01  1.340000e+02  1.510000e+02   
mean                 BARENTS  1.319749e-08  1.093722e-03  3.089055e-03   
                     BATS     1.373881e-09  1.352807e-04  2.706443e-04   
                     GUAM     5.411974e-10  4.401892e-05  8.808126e-05   
                     HOT      1.228245e-09  1.096010e-04  2.193919e-04   
                     PAPA     1.053651e-08  1.157039e-03  3.408163e-03   
variance             BARENTS  2.086815e-16  7.826598e-07  4.308934e-06   
                     BATS     4.891095e-19  5.278132e-09  2.109966e-08   
                     GUAM     5.489673e-21  3.644009e-11  1.457977e-10   
                     HOT      7.045748e-20  6.490739e-10  2.595943e-09   
                     PAPA     3.365107e-17  1.314578e-07  8.362901e-07   

                                       75%         max  
quantity_of_interest station                            
argmax               BARENTS  2.130000e+02  321.000000  
                     BATS     1.000000e+02  365.000000  
                     GUAM     1.960000e+02  365.000000  
                     HOT      6.800000e+01  365.000000  
                     PAPA     2.870000e+02  365.000000  
mean                 BARENTS  7.170793e-03    0.036820  
                     BATS     1.685395e-03    0.089212  
                     GUAM     2.918673e-04    0.036947  
                     HOT      1.058672e-03    0.086528  
                     PAPA     9.263482e-03    0.073865  
variance             BARENTS  1.337905e-05    0.000100  
                     BATS     7.096709e-07    0.000135  
                     GUAM     9.177534e-10    0.000016  
                     HOT      5.378909e-08    0.000065  
                     PAPA     2.977531e-06    0.000114

In [4]:
quantity_of_interest = ["_".join(col).strip() for col in cost_function_results.columns.values]
quantity_of_interest

['BARENTS_mean',
 'BARENTS_variance',
 'BARENTS_argmax',
 'HOT_mean',
 'HOT_variance',
 'HOT_argmax',
 'BATS_mean',
 'BATS_variance',
 'BATS_argmax',
 'PAPA_mean',
 'PAPA_variance',
 'PAPA_argmax',
 'GUAM_mean',
 'GUAM_variance',
 'GUAM_argmax']

In [5]:
# Get unique value but keep order
stations = cost_function_results.columns.levels[0]
quantity_of_interest_function = cost_function_results.columns.levels[1]
print(stations)
print(quantity_of_interest_function)

Index(['BARENTS', 'BATS', 'GUAM', 'HOT', 'PAPA'], dtype='object', name='station')
Index(['argmax', 'mean', 'variance'], dtype='object', name='quantity_of_interest')


In [6]:
parameters_name = [
    "energy_transfert",
    "tr_0",
    "gamma_tr",
    "lambda_0",
    "gamma_lambda",
]

In [7]:
coupled_parameters_name = [
    f"({i}, {j})"
    for i, j in product(parameters_name, parameters_name)
    if parameters_name.index(i) < parameters_name.index(j)
]

In [8]:
sp = ProblemSpec(
    {
        "names": parameters_name,
        "groups": None,
        "bounds": [
            [0, 1],
            [0.1, 50],
            [-0.5, -0.0001],
            [0.1, 500],
            [-0.5, -0.0001],
        ],
        "outputs": quantity_of_interest,
    }
)

In [9]:
sp.set_results(cost_function_results.to_numpy()[: int(np.floor(np.sqrt(55000)) ** 2), ...])

Outputs:
	15 outputs: ['BARENTS_mean', 'BARENTS_variance', 'BARENTS_argmax', 'HOT_mean', 'HOT_variance', 'HOT_argmax', 'BATS_mean', 'BATS_variance', 'BATS_argmax', 'PAPA_mean', 'PAPA_variance', 'PAPA_argmax', 'GUAM_mean', 'GUAM_variance', 'GUAM_argmax']
	54756 evaluations


In [10]:
sp.analyze_sobol()

sobol_results = sp.to_df()

/Users/adm-lehodey/Documents/Workspace/Projects/Seapopym-optimisation/.venv/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


---

# Plotting the Sobol results


In [11]:
def plot_total_analysis(sobol: ProblemSpec):
    figures = []
    for qoi in sobol.analysis:
        fig = make_subplots(
            rows=1, cols=3, subplot_titles=("1st-order Sobol index", "2nd-order Sobol index", "Total Sobol index")
        )
        fig.add_trace(
            go.Bar(
                x=parameters_name,
                y=sobol.analysis[qoi]["S1"],
                error_y={"type": "data", "array": sobol.analysis[qoi]["S1_conf"]},
                name="1st-order Sobol index",
            ),
            row=1,
            col=1,
        )
        fig.add_trace(
            go.Bar(
                x=coupled_parameters_name,
                y=list(filter(np.isfinite, sobol.analysis[qoi]["S2"].flatten())),
                error_y={"type": "data", "array": list(filter(np.isfinite, sobol.analysis[qoi]["S2_conf"].flatten()))},
                name="2nd-order Sobol index",
            ),
            row=1,
            col=2,
        )
        fig.add_trace(
            go.Bar(
                x=parameters_name,
                y=sobol.analysis[qoi]["ST"],
                error_y={"type": "data", "array": sobol.analysis[qoi]["ST_conf"]},
                name="Total Sobol index",
            ),
            row=1,
            col=3,
        )
        fig.update_layout(showlegend=False, title_text=f"Sobol indices for {qoi}")
        figures.append(fig)
    return figures

In [12]:
for fig in plot_total_analysis(sp):
    fig.show()

## Total Index


In [13]:
def plot_total_sobol_analysis(sobol: ProblemSpec, stations: np.ndarray, quantity_of_interest_function: np.ndarray):
    figures = []
    for st in stations:
        fig = make_subplots(rows=1, cols=3, subplot_titles=(quantity_of_interest_function))
        for i, qoif in enumerate(quantity_of_interest_function):
            fig.add_trace(
                go.Bar(
                    x=parameters_name,
                    y=sobol.analysis[f"{st}_{qoif}"]["ST"],
                    error_y={"type": "data", "array": sobol.analysis[f"{st}_{qoif}"]["ST_conf"]},
                    name=qoif,
                ),
                row=1,
                col=i + 1,
            )

            fig.update_layout(showlegend=False, title_text=f"Total Sobol indices for {st}")
        figures.append(fig)
    return figures

In [14]:
for fig in plot_total_sobol_analysis(sp, stations, quantity_of_interest_function):
    fig.show()